In [ ]:
%cd ..
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyrender
import os
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from scipy import optimize
from math import sin, cos
from numpy.linalg import det

from face_reconstruction.data.biwi import BiwiDataLoader
from face_reconstruction.data.iphone import IPhoneDataLoader
from face_reconstruction.model import BaselFaceModel
from face_reconstruction.landmarks import load_bfm_landmarks, detect_landmarks
from face_reconstruction.graphics import draw_pixels_to_image, register_rgb_depth, backproject_points, interpolate_around, SimpleImageRenderer, setup_standard_scene, get_perspective_camera, backproject_image
from face_reconstruction.optim import BFMOptimization, run_icp, NearestNeighborMode, DistanceType, nearest_neighbors
from face_reconstruction.utils.math import add_column, geometric_median
from env import PLOTS_PATH

# 1. Face Model

In [ ]:
bfm = BaselFaceModel.from_h5("model2019_face12.h5")
bfm_landmarks = load_bfm_landmarks("model2019_face12_landmarks_v2")
bfm_landmark_indices = np.array(list(bfm_landmarks.values()))

In [ ]:
n_shape_coefficients = bfm.get_n_shape_coefficients()
n_expression_coefficients = bfm.get_n_expression_coefficients()
n_color_coefficients = bfm.get_n_color_coefficients()

In [ ]:
def get_data(frame_id):
    loader = IPhoneDataLoader()
    frame = loader.get_frame(frame_id)
    img = frame.get_color_image()
    depth_img = frame.get_depth_image()
    img_width = loader.get_image_width()
    img_height = loader.get_image_height()
    intrinsics = frame.get_intrinsics()
    
    depth_threshold = 0.5 # Drop all points behind that threshold
    
    points = backproject_image(intrinsics, depth_img)
    points_to_render = points[:, :3]
    points_to_render *= 1000 # meter to millimeter
    colors = img.reshape(-1, 3)  # Just flatten color image
    
    foreground_mask = depth_img.reshape(-1) < depth_threshold
    pointcloud = points_to_render[foreground_mask]
    colors = colors[foreground_mask]
    
    return img, depth_img, img_width, img_height, intrinsics, pointcloud, colors

# 2. Detect 3D landmarks

In [ ]:
def detect_3d_landmarks(img, depth_img):
    landmarks_img, face_pos = detect_landmarks(img, return_face_pos=True)
    face_pos = face_pos[0]
    
    # Interpolate
    interpolation_size = 1
    rgb_depth_values = [interpolate_around(depth_img, pixel, interpolation_size) for pixel in landmarks_img]
    
    landmark_points_3d = backproject_points(intrinsics, rgb_depth_values, landmarks_img)
    landmark_points_3d_render = np.array(landmark_points_3d)
    landmark_points_3d_render[:,2] = -landmark_points_3d_render[:,2]  # Invert z-coordinate for easier rendering (landmarks will be right in front of camera)if isinstance(loader, IPhoneDataLoader):
    landmark_points_3d_render *= 1000  # meter to millimeter
    
    # It can happen that depth information is bad and back-projected landmark points are far away from the other. These should be ignored
    landmark_points_3d_median = geometric_median(landmark_points_3d_render)
    distances_from_median = np.linalg.norm(landmark_points_3d_render - landmark_points_3d_median, axis=1)
    threshold_landmark_deviation = 500  
    valid_landmark_points_3d = np.where((np.array(rgb_depth_values) != 0) & (distances_from_median < threshold_landmark_deviation))[0]
    
    return face_pos, valid_landmark_points_3d, landmark_points_3d_render

## 2.1 Restrict Pointcloud to Facial Points

In [ ]:
def extract_facial_points(face_pos, depth_img):
    face_depth_values = []
    face_pixels = []
    interpolation_size = 1
    
    for x in range(face_pos.left(), face_pos.right() + 1):
        for y in range(face_pos.top(), face_pos.bottom() + 1):
            pixel = [x, y]
            face_depth_value = interpolate_around(depth_img, pixel, interpolation_size)
            if face_depth_value > 0:
                face_depth_values.append(face_depth_value)
                face_pixels.append(pixel)
                
    face_pointcloud = backproject_points(intrinsics, face_depth_values, face_pixels)
    face_pointcloud[:, 2] = -face_pointcloud[:, 2]
    face_pointcloud_colors = np.array([img[y, x] for x, y in face_pixels])
    face_pointcloud *= 1000  # Meters to Millimeters
    
    body_depth_values = []
    body_pixels = []
    for x in range(img_width):
        for y in range(img_height):
            if (x < face_pos.left() or x > face_pos.right()) or (y < face_pos.top() or y > face_pos.bottom()):
                pixel = [x, y]
                body_depth_value = interpolate_around(depth_img, pixel, interpolation_size)
                if body_depth_value > 0:
                    body_depth_values.append(body_depth_value)
                    body_pixels.append(pixel)
                    
    body_pointcloud = backproject_points(intrinsics, body_depth_values, body_pixels)
    body_pointcloud[:, 2] = -body_pointcloud[:, 2]
    body_pointcloud_colors = np.array([img[y, x] for x, y in body_pixels])
    body_pointcloud *= 1000  # Meters to Millimetersz
    
    return face_pointcloud, body_pointcloud

# 3. ICP

## 3.1 Sparse Recontruction

In [ ]:
n_params_shape_sparse = 3 # 20
n_params_expression_sparse = 3 # 10
weight_shape_params_sparse = 100 # 10000
weight_expression_params_sparse = 100 # 1000
l2_regularization_sparse = 10000  # regularizes only face model parameters
rotation_mode = 'lie'

sparse_optimizer = BFMOptimization(bfm, 
                               n_params_shape=n_params_shape_sparse,
                               n_params_expression=n_params_expression_sparse, 
                               weight_shape_params=weight_shape_params_sparse, 
                               weight_expression_params=weight_expression_params_sparse,
                               rotation_mode=rotation_mode)

In [ ]:
def sparse_recon(landmark_points_3d_render):
    initial_camera_pose = np.eye(4) # position camera just in front of face

    if sparse_optimizer.rotation_mode == 'lie':
        theta = 0.0001
        initial_camera_pose[:3, :3] = np.array([[1, 0, 0], [0, cos(theta), -sin(theta)], [0, sin(theta), cos(theta)]])
        assert abs(det(initial_camera_pose) - 1.0) < 0.00001 

    initial_params = sparse_optimizer.create_parameters(
        [0 for _ in range(n_shape_coefficients)],
        [0 for _ in range(n_expression_coefficients)],
        initial_camera_pose
    )
    
    sparse_loss = sparse_optimizer.create_sparse_loss_3d(bfm_landmark_indices[valid_landmark_points_3d], landmark_points_3d_render[valid_landmark_points_3d], regularization_strength=l2_regularization_sparse)
    sparse_context = sparse_optimizer.create_optimization_context(sparse_loss, initial_params)
    result = sparse_context.run_optimization(sparse_loss, initial_params)
    
    return sparse_context.create_parameters_from_theta(result.x)
    

## 3.2 Dense Reconstruction

In [ ]:
n_params_shape_dense = 20 # 20
n_params_expression_dense = 20 # 10
weight_shape_params_dense = 100 # 10000
weight_expression_params_dense = 100 # 1000
rotation_mode = 'lie'

In [ ]:
dense_optimizer = BFMOptimization(bfm, 
                               n_params_shape=n_params_shape_dense,
                               n_params_expression=n_params_expression_dense, 
                               weight_shape_params=weight_shape_params_dense, 
                               weight_expression_params=weight_expression_params_dense,
                               rotation_mode=rotation_mode)

In [ ]:
def dense_recon(face_pointcloud, params_sparse):
    nn_mode = NearestNeighborMode.FACE_VERTICES # FACE_VERTICES: every face vertex will be assigned its nearest neighbor in pointcloud
                                            # POINTCLOUD: every point in pointcloud will be assigned its nearest neighbor in face model
    distance_type = DistanceType.POINT_TO_POINT
    icp_iterations = 2
    optimization_steps_per_iteration = 20
    l2_regularization_dense = 100 # 100
    
    params, distances, _ = run_icp(dense_optimizer, 
                               face_pointcloud,
                               bfm, 
                               params_sparse.with_new_manager(dense_optimizer), 
                               max_iterations=icp_iterations, 
                               nearest_neighbor_mode=nn_mode, 
                               distance_type=distance_type,
                               max_nfev=optimization_steps_per_iteration,
                               l2_regularization=l2_regularization_dense)
    return params

# 4. Render Face Reconstruction

In [ ]:
def setup_scene(params_render, intrinsics, img_width, img_height, pointcloud, colors, show_pointcloud=True, show_mask=True, show_pointcloud_face=False, cut_around_face=4):
    
    face_mesh = bfm.draw_sample(
        shape_coefficients=params_render.shape_coefficients, 
        expression_coefficients=params_render.expression_coefficients, 
        color_coefficients=params_render.color_coefficients)
    
    bfm_vertices = params_render.camera_pose @ add_column(face_mesh.vertices, 1).T
    distances, indices = nearest_neighbors(pointcloud, bfm_vertices[:3, :].T)
    pointcloud_mask = distances > cut_around_face
    
    perspective_camera = get_perspective_camera(intrinsics, img_width, img_height)
    scene = setup_standard_scene(perspective_camera)
    if show_pointcloud and show_pointcloud_face:
        scene.add(pyrender.Mesh.from_points(pointcloud[pointcloud_mask], colors=colors[pointcloud_mask]), pose=initial_camera_pose)
    if show_mask:
        scene.add(pyrender.Mesh.from_trimesh(bfm.convert_to_trimesh(face_mesh)), pose=params_render.camera_pose)
    if not show_pointcloud and show_pointcloud_face:
        scene.add(pyrender.Mesh.from_points(face_pointcloud, colors=face_pointcloud_colors), pose=initial_camera_pose)
    if show_pointcloud and not show_pointcloud_face:
        scene.add(pyrender.Mesh.from_points(body_pointcloud, colors=body_pointcloud_colors), pose=initial_camera_pose)
    return scene

In [ ]:
def render_img(scene, width, height):
    r = pyrender.OffscreenRenderer(img_width, img_height)
    color, depth = r.render(scene)
    r.delete()
    img_with_mask = np.array(img)
    img_with_mask[depth != 0] = color[depth != 0]
    return img_with_mask

# Loop

In [ ]:
for i in tqdm(range(116)):
    img, depth_img, img_width, img_height, intrinsics, pointcloud, colors = get_data(i)
    face_pos, valid_landmark_points_3d, landmark_points_3d_render = detect_3d_landmarks(img, depth_img)
    face_pointcloud, body_pointcloud = extract_facial_points(face_pos, depth_img)
    params_sparse = sparse_recon(landmark_points_3d_render)
    params_render = dense_recon(face_pointcloud, params_sparse)
    
    scene = setup_scene(params_render, intrinsics, img_width, img_height, pointcloud, colors, show_pointcloud=False, show_mask=True)
    img = render_img(scene, img_width, img_height) 
    
    img_path = os.path.join(PLOTS_PATH, f'marc_{i:03d}.jpg')
    plt.imsave(img_path, img)